<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/GPT4_x_alpaca_13b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sources :**

https://www.listendata.com/2023/04/open-source-gpt-4-models-made-easy.html

https://www.listendata.com/2023/03/open-source-chatgpt-models-step-by-step.html

In [1]:
from IPython.display import clear_output
#@title Connect runtime and mount Google Drive
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

from google.colab import drive
drive.mount('/content/drive')

Tesla T4, 15360 MiB, 15101 MiB
Mounted at /content/drive


In [2]:
%cd /content

/content


## Install Text Generation WebUI

In [3]:
!git clone https://github.com/oobabooga/text-generation-webui
%cd text-generation-webui
!pip install -r requirements.txt

Cloning into 'text-generation-webui'...
remote: Enumerating objects: 8418, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 8418 (delta 406), reused 554 (delta 362), pack-reused 7784
Receiving objects: 100% (8418/8418), 2.89 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (5618/5618), done.
/content/text-generation-webui
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft (to revision e45529b149c7f91ec1d4d82a5a152ef56c56cb94) to /tmp/pip-req-build-a8hi3jw7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-a8hi3jw7
  Running command git rev-parse -q --verify 'sha^e45529b149c7f91ec1d4d82a5a152ef56c56cb94'
  Running command git fetch -q https://github.com/huggingface/peft e45529b149c7f91ec1d4d82a5a152ef56c56cb94
  Running command git checkout -q e45529b149c7f91ec1d4d82a5

## 4-bit Mode Support Setup

Create Repositories folder and save GPTQ-for-LLAMA repos (4 bits quantization of LLaMA using GPTQ)

In [4]:
%mkdir /content/text-generation-webui/repositories/
%cd /content/text-generation-webui/repositories/
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!pip install ninja
!pip install -r requirements.txt
!python setup_cuda.py install

/content/text-generation-webui/repositories
Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 818, done.
remote: Counting objects: 100% (818/818), done.
remote: Compressing objects: 100% (323/323), done.
remote: Total 818 (delta 496), reused 798 (delta 490), pack-reused 0
Receiving objects: 100% (818/818), 471.91 KiB | 2.37 MiB/s, done.
Resolving deltas: 100% (496/496), done.
/content/text-generation-webui/repositories/GPTQ-for-LLaMa
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (145 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

## Download Model

For **gpt4-x-alpaca-13b**

*   anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g
*   https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g/resolve/main/gpt-x-alpaca-13b-native-4bit-128g-cuda.pt

For **vicuna-13b**

*   anon8231489123/vicuna-13b-GPTQ-4bit-128g
*   https://huggingface.co/anon8231489123/vicuna-13b-GPTQ-4bit-128g/resolve/main/vicuna-13b-4bit-128g.safetensors


In [5]:
%cd /content/text-generation-webui/
!python download-model.py --text-only anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g
!wget https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g/resolve/main/gpt-x-alpaca-13b-native-4bit-128g-cuda.pt


/content/text-generation-webui
100% 1.03k/1.03k [00:00<00:00, 8.26MiB/s]
100% 535/535 [00:00<00:00, 4.60MiB/s]
100% 137/137 [00:00<00:00, 1.00MiB/s]
100% 1.05k/1.05k [00:00<00:00, 7.23MiB/s]
100% 33.4k/33.4k [00:00<00:00, 59.7MiB/s]
100% 96.0/96.0 [00:00<00:00, 637kiB/s]
100% 500k/500k [00:00<00:00, 32.0MiB/s]
100% 329/329 [00:00<00:00, 2.69MiB/s]
100% 101k/101k [00:00<00:00, 589kiB/s] 
--2023-06-15 07:35:49--  https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g/resolve/main/gpt-x-alpaca-13b-native-4bit-128g-cuda.pt
Resolving huggingface.co (huggingface.co)... 65.9.17.44, 65.9.17.74, 65.9.17.72, ...
Connecting to huggingface.co (huggingface.co)|65.9.17.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/27/0a/270ae466c5c9f563f4b8b85348f754ed9577a021f88e1be301e7492306fa5cb0/d04313b9191d54f4c6c6e017e63ca79d4eb6b1e0d3588a3923d36f3ace7fa25b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27

**Move .pt or .safetensors file to /models/...**

In [6]:
!mv /content/text-generation-webui/gpt-x-alpaca-13b-native-4bit-128g-cuda.pt /content/text-generation-webui/models/anon8231489123_gpt4-x-alpaca-13b-native-4bit-128g/gpt-x-alpaca-13b-native-4bit-128g-cuda.pt

## Run Application

In [1]:
# To fix this error - AttributeError: module 'PIL.Image' has no attribute 'Resampling'
# Restart runtime after this
!pip install --ignore-installed Pillow==9.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.3 MB)


In [ ]:
%cd /content/text-generation-webui
!python server.py --share --model anon8231489123_gpt4-x-alpaca-13b-native-4bit-128g --model_type llama --chat --wbits 4 --groupsize 128

/content/text-generation-webui
2023-06-15 07:44:53 WARNING:The gradio "share link" feature uses a proprietary executable to create a reverse tunnel. Use it with care.
2023-06-15 07:44:55.491450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
2023-06-15 07:44:58 INFO:Loading anon8231489123_gpt4-x-alpaca-13b-native-4bit-128g...
2023-06-15 07:44:58 INFO:The AutoGPTQ params are: {'model_basename': 'gpt-x-alpaca-13b-native-4bit-128g-cuda', 'device': 'cuda:0', 'use_triton': False, 'use_safetensors': False, 'trust_remote_code': False, 'max_memory': None, 'quantize_config': BaseQuantizeConfig(bits=4, group_size=128, damp_percent=0.01, desc_act=False, sym=True, true_sequential=True, model_name_or_path=None, model_file_base_name=None)}
2023-06-15 07:44:59 WARNING:CUDA extension not installed.
2023-06-15 07:45:29 WARNING:skip module injection for FusedLlamaMLPForQua